In [334]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import api_keys
from spotipy.oauth2 import SpotifyOAuth
import os
import requests
from dotenv import load_dotenv
import base64
import json
import webbrowser
import urllib

## https://developer.spotify.com/documentation/web-api/reference/get-users-saved-tracks

In [335]:
load_dotenv()
REDIRECT_URI = "http://localhost"
CLIENT_ID = os.getenv("client_id")
CLIENT_SECRET = os.getenv("client_secret")
SCOPE = 'user-library-read'

## Request User Authorization

In [336]:
# Create the authorization URL
params = {
    'response_type': 'code',
    'client_id': CLIENT_ID,
    'scope': SCOPE,
    'redirect_uri': REDIRECT_URI,
}
url = f"https://accounts.spotify.com/authorize?{urllib.parse.urlencode(params)}"

# Open the authorization URL in the default web browser
webbrowser.open(url)

True

In [337]:
CODE = "AQDuLH1MuvdUMxRsP5ZBM8nXk-qCBezONRjJYudVbrqZIanKIf6mK5kSVyx15685cNuVS1kid4Lr07irDUtjBsHk4tjspwY8yv1yzF6F1A2fYXDUpvwa5zNDMOSSpVYiLbOUQF5qw8yn-HvVOdtL1xcYo9AU7qoe8v6osmXV0UbGaJe6q5Ek"

## Get Token Info
- Lasts only 1 hour

In [338]:
def get_token_info():
    auth_header = base64.b64encode(f"{CLIENT_ID}:{CLIENT_SECRET}".encode('utf-8')).decode('utf-8')
    # auth_string = client_id + ":" + client_secret
    # auth_bytes = auth_string.encode("utf-8")
    # auth_base64 = str(base64.b64encode(auth_bytes): "utf-8" )

    url = "https://accounts.spotify.com/api/token"
    headers = {
            "Authorization": f"Basic {auth_header}",
            "Content-Type": "application/x-www-form-urlencoded",
    }
    data = {
            "grant_type": "authorization_code",
            "code" : CODE,
            "redirect_uri": REDIRECT_URI}
    result = requests.post(url, headers=headers, data=data)
    json_results = json.loads(result.content)
    token = json_results["access_token"]
    refresh_token = json_results["refresh_token"]
    return json_results

token_info = get_token_info()

In [339]:
token_info

{'access_token': 'BQAxBim6k7DyejGGGDML2mWAbu0LDsumDcBWNG-cAhXNZn5K1vPWRepLLdoNgJhX34qb4aGh6XXUQjNHwzUAPiWpb4hfFlcLEnKrbguWexrl5lODFqtFXUWJ3Vzcujlt8LHOhrGU8erwzPt_BG18smjpOWVQiyTAwP6MJAJSt_8hAiibqXpy7XH3hpwqaPA',
 'token_type': 'Bearer',
 'expires_in': 3600,
 'refresh_token': 'AQBi79pyK9ME2j9U8VRMy5Ay_xT9iNimTuavvAlWnoPR_NAjvsg4jr01Bxizahr122rlQfNSayaYDsY9yTt6Y7-YFtvC7GgNHaPd3Sq564TjiH8C8ksTpBnMD7ymQlZKSyc',
 'scope': 'user-library-read'}

In [340]:
token = token_info["access_token"]
refresh_token = token_info["refresh_token"]
scope = token_info["scope"]

## Getting Refresh Token

In [341]:
# def authorize():
#     # The scopes your application needs access to
#     scopes = ['user-library-read']

#     # Create the authorization URL
#     params = {
#         'client_id': client_id,
#         'response_type': 'code',
#         'redirect_uri': REDIRECT_URI,
#         'scope': ' '.join(scopes)

## Refresh Token

In [342]:

# client_id = os.getenv("client_id")
# client_secret = os.getenv("client_secret")

# def get_token():
#     auth_string = f"{client_id}:{client_secret}"
#     auth_bytes = auth_string.encode("utf-8")
#     auth_base64 = str(base64.b64encode(auth_bytes), "utf-8" )

#     url = "https://accounts.spotify.com/api/token"
#     headers = {
#             "Authorization": f"Basic {auth_base64}",
#             "Content-Type": "application/x-www-form-urlencoded"
#     }
#     data = {"grant_type": "refresh_token"}
#     result = post(url, headers=headers, data=data)
#     json_results = json.loads(result.content)
#     token = json_results["access_token"]
#     refresh_token = json_results["refresh_token"]
#     return token, refresh_token

# refresh_token = get_token()

## Get Auth Header

In [343]:
def get_auth_header(token):
    return f"Bearer " + token

In [347]:
get_auth_header(token)

'Bearer BQAxBim6k7DyejGGGDML2mWAbu0LDsumDcBWNG-cAhXNZn5K1vPWRepLLdoNgJhX34qb4aGh6XXUQjNHwzUAPiWpb4hfFlcLEnKrbguWexrl5lODFqtFXUWJ3Vzcujlt8LHOhrGU8erwzPt_BG18smjpOWVQiyTAwP6MJAJSt_8hAiibqXpy7XH3hpwqaPA'

## Post Request
- Request using token for API endpoints

In [348]:
SPOTIFY_GET_CURRENT_USER_SAVED_TRACKS_URL = "https://api.spotify.com/v1/me/tracks?limit=50&offset=0"
SPOTIFY_GET_CURRENT_TRACK_URL = 'https://api.spotify.com/v1/me/player/currently-playing'

In [349]:
def get_current_user_tracks(url):
    response = requests.get(url=url, headers={"Authorization" : f"{get_auth_header(token)}"})
    resp_json = response.json()
    return resp_json

In [350]:
def loop_through():
    first_spotify_url = "https://api.spotify.com/v1/me/tracks?limit=50&offset=0"
    results = get_current_user_tracks(first_spotify_url)
    # increment_of50_list = [i for i in range(0, get_current_user_tracks()["total"], 50)]
    # increment_of50_list_test = [i for i in range(0, int(get_current_user_tracks(SPOTIFY_GET_CURRENT_USER_SAVED_TRACKS_URL)["total"]) - 4300, 50)]
    full_list = []
    current_url = results["href"]
    # while results["next"] is not None:
    for i in range(4):
        results = get_current_user_tracks(current_url)
        full_list.append(results)
        spotify_url_next = results["next"]
        current_url = spotify_url_next
    return full_list
            

In [351]:
test_results = loop_through()

## Results

In [352]:
# full_list_parsed = []
full_list_parsed={}
for index_i, i in enumerate(test_results):  
    for index_j, j in enumerate(test_results[index_i]["items"]):
        # track_details = {}
        full_list_parsed[j["track"]["uri"]] = {
              "added_to_playlist_time":j["added_at"]
            , "artist_name":j["track"]["album"]["artists"][0]["name"]
            , "unique_artist_id": j["track"]["album"]["artists"][0]["uri"]
            , "release_date": j["track"]["album"]["release_date"]
            , "song_name": j["track"]["name"]
            , "song_length_seconds": str(int(j["track"]["duration_ms"])/1000)
            , "song_explicit": str(j["track"]["explicit"])
            , "song_linkId": j["track"]["external_urls"]["spotify"]
            , "song_popularity": str(j["track"]["popularity"])
            , "album_name": j["track"]["album"]["name"]
            , "spotify_uri_album": j["track"]["album"]["uri"]
            , "spotify_track_id": j["track"]["id"]  
                                            }
        # full_list_parsed.append(track_details)
        # print(index_i, j)

In [353]:
full_list_parsed

{'spotify:track:4p16E9c9Ig6xFMGS3Y82mT': {'added_to_playlist_time': '2023-06-09T15:26:19Z',
  'artist_name': 'RL Grime',
  'unique_artist_id': 'spotify:artist:5eIbEEQnDM8yuDVB0bimSP',
  'release_date': '2023-06-09',
  'song_name': 'Pour Your Heart Out (feat. 070 Shake)',
  'song_length_seconds': '223.561',
  'song_explicit': 'False',
  'song_linkId': 'https://open.spotify.com/track/4p16E9c9Ig6xFMGS3Y82mT',
  'song_popularity': '67',
  'album_name': 'Pour Your Heart Out (feat. 070 Shake)',
  'spotify_uri_album': 'spotify:album:3IqciDNeCpyycdfxs6AA6T',
  'spotify_track_id': '4p16E9c9Ig6xFMGS3Y82mT'},
 'spotify:track:3tAzIzJ76pdgn17SIoMSXO': {'added_to_playlist_time': '2023-06-08T15:48:43Z',
  'artist_name': 'NURKO',
  'unique_artist_id': 'spotify:artist:757FXqX0Osk2pqtgv4E5v4',
  'release_date': '2019-01-31',
  'song_name': 'Breathe Without',
  'song_length_seconds': '226.315',
  'song_explicit': 'False',
  'song_linkId': 'https://open.spotify.com/track/3tAzIzJ76pdgn17SIoMSXO',
  'song_p

## Get song genres from Get Track API Endpoint
### Seems as if most songs do not have a genre object/attribute associated with them (shelving this for now)

In [226]:
# for i in full_list_parsed[0:10]:
#     print(i[0])

# full_list_parsed[0]["0.0"][-1].split(":")[-1]
# list(full_list_parsed[0].keys())

In [224]:
def get_track_genres(url):
    response = requests.get(url=url, headers={"Authorization" : f"{get_auth_header(token)}"})
    resp_json = response.json()
    return resp_json


get_track_genres("https://api.spotify.com/v1/tracks?ids=6POZ7ocsOQQvJrfArz6KOD")
# for track_id in full_list_parsed:
#     get_track_genre_url= f"https://api.spotify.com/v1/tracks/{track_id}"
#     get_track_genres(get_track_genre_url)

{'error': {'status': 401, 'message': 'The access token expired'}}

## Call Track Auto Features API endpoint
- https://developer.spotify.com/documentation/web-api/reference/get-several-audio-features

In [354]:
full_list_parsed

{'spotify:track:4p16E9c9Ig6xFMGS3Y82mT': {'added_to_playlist_time': '2023-06-09T15:26:19Z',
  'artist_name': 'RL Grime',
  'unique_artist_id': 'spotify:artist:5eIbEEQnDM8yuDVB0bimSP',
  'release_date': '2023-06-09',
  'song_name': 'Pour Your Heart Out (feat. 070 Shake)',
  'song_length_seconds': '223.561',
  'song_explicit': 'False',
  'song_linkId': 'https://open.spotify.com/track/4p16E9c9Ig6xFMGS3Y82mT',
  'song_popularity': '67',
  'album_name': 'Pour Your Heart Out (feat. 070 Shake)',
  'spotify_uri_album': 'spotify:album:3IqciDNeCpyycdfxs6AA6T',
  'spotify_track_id': '4p16E9c9Ig6xFMGS3Y82mT'},
 'spotify:track:3tAzIzJ76pdgn17SIoMSXO': {'added_to_playlist_time': '2023-06-08T15:48:43Z',
  'artist_name': 'NURKO',
  'unique_artist_id': 'spotify:artist:757FXqX0Osk2pqtgv4E5v4',
  'release_date': '2019-01-31',
  'song_name': 'Breathe Without',
  'song_length_seconds': '226.315',
  'song_explicit': 'False',
  'song_linkId': 'https://open.spotify.com/track/3tAzIzJ76pdgn17SIoMSXO',
  'song_p

In [355]:
def get_tracks_audio_analysis(url):
    response = requests.get(url=url, headers={"Authorization" : f"{get_auth_header(token)}"})
    resp_json = response.json()
    return resp_json

In [356]:
listtemp=[]
# start_index=0
for index, i in enumerate(list(full_list_parsed.values())):
    listtemp.append(i["spotify_track_id"])

In [357]:
listtemp

['4p16E9c9Ig6xFMGS3Y82mT',
 '3tAzIzJ76pdgn17SIoMSXO',
 '68eKAppHA0LgssmPmoRNv4',
 '5UNtPhsi8vPmJI3yeaiKTS',
 '6POZ7ocsOQQvJrfArz6KOD',
 '7dc5v0lbm5rA9XAPSixTL6',
 '50x1Ic8CaXkYNvjmxe3WXy',
 '1xSmjB0t19q4CSJ0X0ymGB',
 '3jIUcNxBowyf8clrQ9bPIy',
 '0gWrMbx6pbdH3n3nsLjE55',
 '2UbdtdpOW3URCTqKsPATJg',
 '0TfMitxzrMd2NTLxNXL2Hu',
 '6qyx2UzpeviUgXBeuCbZR0',
 '5gmqIUiSsnADV8bpOKFX0H',
 '3BINSFSuHalV8yoX3zAVbl',
 '2IhXds0wZkbklKo12XGhb4',
 '2qUPj0b3PYLJJoscQvhMUr',
 '6Sw0DXMinATz4FTYaO9MxS',
 '1TFljRRw6jLg0qGP1Qa1Gc',
 '0SGC6p2Vl7AYKYWtbV1jIG',
 '5W4kiM2cUYBJXKRudNyxjW',
 '17sPXQa8bT24ToCYHXuQXI',
 '6VKNW0svDbddkdNDGAE0NY',
 '62PCV9ANdpPd9FhWgNljeW',
 '526fD9LiAEi3KKvhhYfWmm',
 '1N8klHEbZ0bFOHnMVs8C9S',
 '3imYkN34og7muZbkSZWLUW',
 '5DBD1TTl06dvp3WqPEY5Pj',
 '5jUOG0aeAofL5u3UWyrtDO',
 '2rxQMGVafnNaRaXlRMWPde',
 '3U1V2JWnoLV7wVYgOPPDBR',
 '4otlvyelfJJxZlu63beJUN',
 '6c12PXXRn68whIxd3AWf2y',
 '0judK8TqYfvwUohBngUxrE',
 '6DswwzQlABrNoqjNydtvuX',
 '38KoNvv2irkAq695ZDBmW6',
 '6XXZzGJ0I1EjsGW8wKyGfO',
 

## Breaking up Id Lists
- Track Audio Analysis only takes in 100 ids at a time

In [358]:
def list_chunks(l,n):
    for i in range(0,len(l), n):
        yield l[i:i + n]

n = 100

x=list(list_chunks(listtemp,n))

In [359]:
string_urls=[]
for index,i in enumerate(x):
    string_urls.append(','.join(i))

In [360]:
string_urls

['4p16E9c9Ig6xFMGS3Y82mT,3tAzIzJ76pdgn17SIoMSXO,68eKAppHA0LgssmPmoRNv4,5UNtPhsi8vPmJI3yeaiKTS,6POZ7ocsOQQvJrfArz6KOD,7dc5v0lbm5rA9XAPSixTL6,50x1Ic8CaXkYNvjmxe3WXy,1xSmjB0t19q4CSJ0X0ymGB,3jIUcNxBowyf8clrQ9bPIy,0gWrMbx6pbdH3n3nsLjE55,2UbdtdpOW3URCTqKsPATJg,0TfMitxzrMd2NTLxNXL2Hu,6qyx2UzpeviUgXBeuCbZR0,5gmqIUiSsnADV8bpOKFX0H,3BINSFSuHalV8yoX3zAVbl,2IhXds0wZkbklKo12XGhb4,2qUPj0b3PYLJJoscQvhMUr,6Sw0DXMinATz4FTYaO9MxS,1TFljRRw6jLg0qGP1Qa1Gc,0SGC6p2Vl7AYKYWtbV1jIG,5W4kiM2cUYBJXKRudNyxjW,17sPXQa8bT24ToCYHXuQXI,6VKNW0svDbddkdNDGAE0NY,62PCV9ANdpPd9FhWgNljeW,526fD9LiAEi3KKvhhYfWmm,1N8klHEbZ0bFOHnMVs8C9S,3imYkN34og7muZbkSZWLUW,5DBD1TTl06dvp3WqPEY5Pj,5jUOG0aeAofL5u3UWyrtDO,2rxQMGVafnNaRaXlRMWPde,3U1V2JWnoLV7wVYgOPPDBR,4otlvyelfJJxZlu63beJUN,6c12PXXRn68whIxd3AWf2y,0judK8TqYfvwUohBngUxrE,6DswwzQlABrNoqjNydtvuX,38KoNvv2irkAq695ZDBmW6,6XXZzGJ0I1EjsGW8wKyGfO,3RF00zOzrrWKKoU7aIBaTS,5rIpzFDp0sYDCQytYDG96W,7uSZPbd8DxdkmSauCKnITY,1tdLHkejJyiMiLjSxLFPcQ,7JFj5qHgeTfsKdtQYUYI7B,693xmBAOD4xwkS4W71W1Jn,0i2Rbagsi

In [361]:
results_all=[]
for index,i in enumerate(string_urls):
    results_temp= get_tracks_audio_analysis(f'https://api.spotify.com/v1/audio-features?ids={string_urls[index]}')
    results_all.append(results_temp)
results_all

[{'audio_features': [{'danceability': 0.494,
    'energy': 0.917,
    'key': 7,
    'loudness': -3.302,
    'mode': 0,
    'speechiness': 0.144,
    'acousticness': 0.0381,
    'instrumentalness': 0,
    'liveness': 0.0943,
    'valence': 0.178,
    'tempo': 145.847,
    'type': 'audio_features',
    'id': '4p16E9c9Ig6xFMGS3Y82mT',
    'uri': 'spotify:track:4p16E9c9Ig6xFMGS3Y82mT',
    'track_href': 'https://api.spotify.com/v1/tracks/4p16E9c9Ig6xFMGS3Y82mT',
    'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4p16E9c9Ig6xFMGS3Y82mT',
    'duration_ms': 223562,
    'time_signature': 4},
   {'danceability': 0.455,
    'energy': 0.809,
    'key': 8,
    'loudness': -3.333,
    'mode': 1,
    'speechiness': 0.0528,
    'acousticness': 0.524,
    'instrumentalness': 2.97e-06,
    'liveness': 0.137,
    'valence': 0.201,
    'tempo': 113.638,
    'type': 'audio_features',
    'id': '3tAzIzJ76pdgn17SIoMSXO',
    'uri': 'spotify:track:3tAzIzJ76pdgn17SIoMSXO',
    'track_href': 'http

In [319]:
results_all

[]

In [367]:
track_audio_features={}
for i in results_all:
    for j in i["audio_features"]:
        track_audio_features[j['uri']] ={
                "danceability":j["danceability"]
                , "energy":j["energy"]
                , "key":j["key"]
                , "loudness":j["loudness"]
                , "mode":j["mode"]
                , "speechiness":j["speechiness"]
                , "acousticness":j["acousticness"]
                , "instrumentalness":j["instrumentalness"]
                , "liveness":j["liveness"]
                , "valence":j["valence"]
                , "time_signature":j["time_signature"]
                , "tempo":j["tempo"]
                                        }

In [368]:
track_audio_features

{'spotify:track:4p16E9c9Ig6xFMGS3Y82mT': {'danceability': 0.494,
  'energy': 0.917,
  'key': 7,
  'loudness': -3.302,
  'mode': 0,
  'speechiness': 0.144,
  'acousticness': 0.0381,
  'instrumentalness': 0,
  'liveness': 0.0943,
  'valence': 0.178,
  'time_signature': 4,
  'tempo': 145.847},
 'spotify:track:3tAzIzJ76pdgn17SIoMSXO': {'danceability': 0.455,
  'energy': 0.809,
  'key': 8,
  'loudness': -3.333,
  'mode': 1,
  'speechiness': 0.0528,
  'acousticness': 0.524,
  'instrumentalness': 2.97e-06,
  'liveness': 0.137,
  'valence': 0.201,
  'time_signature': 4,
  'tempo': 113.638},
 'spotify:track:68eKAppHA0LgssmPmoRNv4': {'danceability': 0.383,
  'energy': 0.687,
  'key': 8,
  'loudness': -5.558,
  'mode': 1,
  'speechiness': 0.0502,
  'acousticness': 0.29,
  'instrumentalness': 0.000402,
  'liveness': 0.104,
  'valence': 0.104,
  'time_signature': 4,
  'tempo': 133.995},
 'spotify:track:5UNtPhsi8vPmJI3yeaiKTS': {'danceability': 0.505,
  'energy': 0.61,
  'key': 5,
  'loudness': -5.6